In [1]:
import os
import sagemaker
from sagemaker.tensorflow import TensorFlowModel

#create model
sagemaker_session = sagemaker.Session()
role = 'arn:aws:iam::712805919848:role/sagemaker-role'
bucket = 'mysagemaker-us-east'
s3_path ='s3://mysagemaker-us-east/sagemaker_model/model.tar.gz'
env = {'SAGEMAKER_TFS_DEFAULT_MODEL_NAME': 'mobilenet_v2_130_224'}
#model_data = sagemaker_session.upload_data(path=s3_path)

# The "Model" object doesn't create a SageMaker Model until a Transform Job or Endpoint is created.
tensorflow_serving_model = TensorFlowModel(model_data=s3_path,
                                 entry_point='inference.py',          
                                 role=role)
#deploy and create end point
print('start deploy')
predictor = tensorflow_serving_model.deploy(initial_instance_count=1,
                                            endpoint_name = 'sagemaker-test-endpoint12',
                                            #framework_version='1.13',
                                            instance_type='ml.t2.medium')
#-------------------------------------------------------------------------------

print('start predict')

from IPython.display import Image
import numpy as np
from keras.preprocessing import image


image_list = ['inputimage.jpeg']
directory = './'
images = np.empty((len(image_list), 299, 299, 3), dtype=np.float32)
# for filename in image_list:
for i,filename in enumerate(image_list):
    path = os.path.join(directory, filename)
    Image(path)
    img = image.load_img(path, target_size=(299, 299))
    x = image.img_to_array(img)
    images[i] = x

print(images.shape)
# to_send = images[-1]  # works for a unique image
data = images  # doesn't work for a batch of images
# some other attempts that did not work
# to_send = images.tolist()
# to_send = [images[0].tolist(), images[1].tolist()]
print(np.shape(data))

predict_response = predictor.predict(data)
print(predict_response)

No handlers could be found for logger "sagemaker"


start deploy
-*

UnexpectedStatusException: Error hosting endpoint sagemaker-test-endpoint11: Failed. Reason:  The image '520713654638.dkr.ecr.us-east-1.amazonaws.com/sagemaker-tensorflow-serving:1.4-cpu' does not exist..